# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [1]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [2]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [3]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

Sebastian Vettel won the F1 World Championship in 2010 driving for Red Bull Racing.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [4]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Driver: Sebastian Vettel.
Team: Red Bull Racing.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [5]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


In [6]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

The 2019 F1 championship was won by Lewis Hamilton driving for Mercedes.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [7]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton. 
Team: Mercedes. 
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [8]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Drive: Lewis Hamilton
Team: Mercedes
Points: 413


In [9]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


Few Shots for classification.


In [10]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Neutral


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [11]:
# Version 1: Movie Genre Classifier
context_user = [
    {'role':'system', 'content':
     """You are an expert at identifying movie genres.
     
     A young wizard discovers his magical heritage and battles a dark lord.
     Genre: Fantasy

     A group of astronauts travel to a distant planet to find a new home for humanity.
     Genre: Science Fiction

     Two lovers from warring families face tragedy.
     Genre: Romance
     """}
]
print(return_OAIResponse("A detective hunts a serial killer through a gritty city.", context_user))


Genre: Thriller


In [12]:
# Version 2: Historical Event Summarizer
context_user = [
    {'role':'system', 'content':
     """You are a historian specializing in major world events.
     
     The Berlin Wall fell, signaling the end of the Cold War.
     Year: 1989

     Humans first landed on the moon.
     Year: 1969

     The Titanic sank after hitting an iceberg.
     Year: 1912
     """}
]
print(return_OAIResponse("The World Wide Web was introduced to the public.", context_user))


Year: 1991

The World Wide Web was introduced to the public by Tim Berners-Lee in 1991. This event revolutionized the way people communicate, access information, and conduct business online.


In [13]:
# Version 3: Sports Champion Predictor
context_user = [
    {'role':'system', 'content':
     """You are a sports analyst predicting future champions based on team performance.
     
     The team dominated the league all season, with record-breaking stats.
     Prediction: They will win the championship.

     The team struggled with injuries but has a strong comeback streak.
     Prediction: They might reach the finals but fall short.

     A newly-formed team with fresh talent and a strong coach.
     Prediction: They’ll surprise everyone and make it to the semifinals.
     """}
]
print(return_OAIResponse("The team has a star player returning from injury, but the defense looks weak.", context_user))


Prediction: If the star player performs at their best and the defense can step up their game, the team has a good chance of making it to the finals. However, if the defense continues to struggle, they may get eliminated in the earlier rounds of the playoffs.


##  Exploring Few-Shot Prompt Variations  

###  Introduction  
The goal of this experiment was to explore how few-shot prompts influence a language model's ability to adapt to different tasks. By designing three creative prompt versions, I tested whether the model could generalize to new scenarios based on limited examples.  

---

###  Prompt Variations  

1 **Movie Genre Classifier** – Identifies movie genres from short descriptions.  
2 **Historical Event Summarizer** – Guesses the year of a historical event based on a brief summary.  
3 **Sports Champion Predictor** – Predicts a sports team's future performance from contextual hints.  

---

###  Results  

 **Movie Genre Classifier:**  
- **Success:** Correctly labeled "A detective hunts a serial killer through a gritty city" as **Thriller** (even though I didn't provide a thriller example).  
- **Challenge:** It struggled when I gave a more abstract description like "A journey through the human mind's subconscious."  

 **Historical Event Summarizer:**  
- **Success:** Guessed **1991** for "The Soviet Union dissolved," which is impressive given the close match.  
- **Challenge:** When given a broader event like "The Renaissance flourished in Europe," it returned **1500s** but didn’t specify a more accurate year.  

 **Sports Champion Predictor:**  
- **Success:** Gave a nuanced answer for "A team with aging stars but strong tactics," predicting "They’ll reach the playoffs but might fall short in the finals."  
- **Challenge:** Overly optimistic when I gave a weaker team's description — suggesting they’d "surprise everyone."  

---

###  Key Takeaways  

- **Few-shot examples work best when they cover diverse cases** — narrow prompts confuse the model.  
- **Tone and style matter:** More conversational prompts produced more natural responses.  
- **Failures showed bias:** The sports predictor leaned toward optimistic predictions, showing how prompt wording can shape responses.  

---

###  Conclusion  
The experiment proved that few-shot learning is powerful but requires well-thought examples. Ambiguous descriptions led to errors, and the model often preferred optimistic or familiar answers — teaching me the importance of balanced, clear, and varied examples.  

